# Outline of thought process
- starting out with the creature class as the base class. The creature class will be used to create monsters/creatures that<br>
have not become npcs

- the pc_class class will be used to give player characters access to class mechanics
- the pc class will inherit from the creature class and the pc_class class to create a pc
- the npc class will inherit from the creature class and set the is_npc flag of an object to True so that a monster can be <br>
changed to npc status and stored to come back as a recurring character
- the item class will be used to create all items(including interactable objects our pcs might run into or use to change the <br> course of a battle. Think setting a fire to obscure the battle field or wedging a door shut


In [1]:
from dice import Dice
from dice import dice_avg
from dice import DiceBag
from skills import skills
import pc_classes
import creatures
import races
import weapons
import copy
from armor import light_armor

In [2]:
### start of a creature class

class Creature():
    def __init__(self, entity_type, creature):
        creature_attrs = creatures.creature_keys
        self.entity_type = entity_type
        if entity_type == 'pc':
            self.disposition = 'ally'
            self.hp_max = 0
        for key in creature.keys():
            setattr(self, key, creature[key])
        for attr in creature_attrs:
            if hasattr(self, attr)==True:
                pass
            else:
                setattr(self, attr, False)
   # def equip_armor(self, armor):
   #     for prof in self.proficiencies:
            
        
            

    #def attack(weapon):
    #def add_inventory:
    #def cast(spell):
        #if spell.r_attack == True:
            #attack(spell)
        #elif spell.r_save == True:
        #else:
            #do a thing
    #def attack(weapon):
        #if weapon.a_type == melee:
            #do some things
        #elif weapon.a_type == ranged:
            #do some things
    #def dash(movement):
    #def disengage():
    #def dodge(self.advantage_toggle):
    #def help(target):
    #def hide(stealth_check):
    #def ready(trigger, action):
    #def search(perception_check):
    #def use_an_object():
    #def imporovise(skill_check):


        


In [3]:
class Pc(Creature):
    def __init__(self):
        pass
    def create_pc(self, creature):
        setattr(creature, 'pc', True)
        setattr(creature, 'pc_lvl', 1)
        setattr(creature, 'proficiency_bonus', pc_classes.prof_bonus[creature.pc_lvl])
        creature.skills = copy.deepcopy(skills)
        if creature.armor == False and creature.ac == False:
            dex_mod = creatures.ability_mod_tbl[creature.abilities['dex']]
            wis_mod = creatures.ability_mod_tbl[creature.abilities['wis']]
            con_mod = creatures.ability_mod_tbl[creature.abilities['con']]
            barb_ua = 10 + con_mod + dex_mod
            monk_ua = 10 + dex_mod + wis_mod
            if 'barbarian' in creature.pc_classes.keys() & 'monk' in creature.pc_classes.keys():
                unarmored_defense = []
                unarmored_defense.append(barb_ua)
                unarmored_defense.append(monk_ua)
                creature.ac = max(unarmored_defense)
            elif 'barbarian' in creature.pc_classes.keys():
                creature.ac = barb_ua
            elif 'monk' in creature.pc_classes.keys():
                creature.ac = monk_ua
            else:
                creature.ac = 10 + creatures.ability_mod_tbl[creature.abilities['dex']]
        

In [4]:
# keeping pc_class and pc as different classes to account for multi-classing

class PcClass:
    def __init__(self):
        pass
    
    def hill_dwarf_hp_bonus(self, creature):
        creature.hp_max += 1
        
    def add_first_class(self, creature, pc_class, gen_method):
        p_class = pc_class['pc_class']
        setattr(creature, 'pc_classes', {p_class:1})
        setattr(creature, 'abilities', pc_class['ability_by_method'][gen_method])
        
        ### adds first class attrs to creature
        for key in pc_class.keys():
            if key=='pc_class':
                pass
            else:
                setattr(creature, key, pc_class[key])
                
        ### adds racial bonuses to ability scores 
        for key in creature.ability_score_increase.keys():
            creature.abilities[key] += creature.ability_score_increase[key]
        Pc.create_pc(self, creature)
        
        ### sets hp_max
        con_bonus = creatures.ability_mod_tbl[creature.abilities['con']]
        hp_max = int(pc_class['hit_die'][1:]) + con_bonus
        creature.hp_max = hp_max
        if creature.subrace == 'hill':
            self.hill_dwarf_hp_bonus(creature)
            
    def add_class_lvl(self, creature, p_class_name):
        ### update hp_max
        class_hp_add = dice_avg[creature.hit_die]
        con_bonus = creatures.ability_mod_tbl[creature.abilities['con']]
        creature.hp_max += class_hp_add + con_bonus
        if creature.subrace == 'hill':
            self.hill_dwarf_hp_bonus(creature)
        
        creature.pc_classes[p_class_name] += 1
        creature.pc_lvl += 1
        creature.proficiency_bonus = pc_classes.prof_bonus[creature.pc_lvl]


In [5]:
class Npc(Creature):
    def __init__(self):
        pass
    def is_npc(self):
        setattr(self, 'npc', True)
        setattr(creature, 'max_hp', 0)

In [6]:
class item:
    def __init__(self):
        pass
    

In [7]:
class Combat:
    def __init__(self, combatants):
        self.round = 1
        self.combatants = combatants
        #self.c_area = c_area
    
#    def initiative(self.combatants):
#        for c in self.combatants:
#            setattr(c, 'initiative', d20() + c.initiative_bonus)
    
        

In [8]:
tiny = Creature('pc', races.dwarf_hill)
pc_class = PcClass()
pc = Pc()

In [9]:
pc_class.add_first_class(tiny, pc_classes.barbarian, 'pb')

In [10]:
tiny.hp_max

16

In [11]:
tiny.pc_classes

{'barbarian': 1}

In [12]:
for i in range(19):
    pc_class.add_class_lvl(tiny, 'barbarian')

In [13]:
tiny.pc_classes['barbarian']

20

In [14]:
tiny.hp_max

225

In [15]:
tiny.proficiency_bonus

6

In [16]:
tiny.ac

15

In [17]:
tiny.pc_classes

{'barbarian': 20}

In [19]:
tiny.pc_lvl

20

In [24]:
for i in races.elf_high['proficiencies']:
    print(i[0])

perception
longsword
shortsword
shortbow
longbow


In [26]:
[key for key in tiny.skills.keys()]

['acrobatics',
 'animal handling',
 'arcana',
 'athletics',
 'deception',
 'history',
 'insight',
 'intimidation',
 'investigation',
 'medicine',
 'nature',
 'perception',
 'performance',
 'persuasion',
 'religion',
 'sleight of hand',
 'stealth',
 'survival']

In [25]:
### consume spell slot command